In [1]:
import pandas as pd
import copy
import networkx as nx
import numpy as np

In [2]:
# The starting position and goal
starting = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]

goal = [
    [3, 14, 1, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 2, 15, 0]
]

In [3]:
# block.csv contains the information for how pieces 5-12 can block each other's movement due to the second layer.
# the first column is the static piece and the second is the piece trying to mave
# the third column is a space separated list of directions of the gap;direction of movement
# for example, "5, 6, r;u" means that piece 5 blocks piece 6 from moving up into a gap to the right of piece 5.

# This code parses the CSV and creates a Pandas DataFrame of the contents
# The direcions (r)ight, (l)eft, (u)p, (d)own and their combinations are converted to x,y pairs.
# For example, r becomes x=1, y=0 and ld becomes x=-1, y=1.

blocks = []

with open('block.csv') as f:
    for l in f.readlines():
        static_piece, moving_piece, dirs = l.split(',')
        for d in dirs.strip().split():
            x = 0
            y = 0
            loc, movement = d.split(';')
            for c in loc:
                if c == 'u':
                    y -= 1
                elif c == 'd': 
                    y += 1
                elif c == 'l':
                    x -= 1
                elif c == 'r':
                    x += 1
                else:
                    raise c
                    
            for c in movement:
                blocks.append((int(static_piece), int(moving_piece), x, y, c))
            
df = pd.DataFrame.from_records(blocks).rename(columns={0: 'static_piece', 1: 'moving_piece', 2: 'x', 3: 'y', 4: 'direction'})
df.head()

,static_piece,moving_piece,x,y,direction
0,5,6,1,0,u
1,5,6,-1,1,d
2,5,6,-1,0,u
3,5,7,1,0,u
4,5,8,1,0,u


In [4]:
# Return the up to 4 adjacent locations to the given space.
def adjacent(space):
    spaces = []
    
    for x,y,d in [(0,1,'u'), (0,-1,'d'), (1,0,'l'), (-1,0,'r')]:
        x = space[0] + x
        y = space[1] + y
        if (x >= 0) and (x < 4) and (y >= 0) and (y < 4):
            spaces.append((x,y,d))
            
    return spaces

# A function that returns the relative location of all the pieces to a given space as a DataFrame
def relative_locations(space, state):
    pieces = []
    for y in range(4):
        for x in range(4):
            pieces.append((state[y][x], space[0] - x, space[1] - y))
            
    return pd.DataFrame.from_records(pieces).rename(
        columns={0: 'static_piece', 1: 'x', 2: 'y'}
    )


# Apply a given to the board and return a new copy after the change.
# Essentially, swap the locations of the gap (0) with the piece and the given location.
def apply_move(state, space_coords, piece_coords):
    piece = state[piece_coords[1]][piece_coords[0]]
    state_copy = copy.deepcopy(state)
    state_copy[space_coords[1]][space_coords[0]] = piece
    state_copy[piece_coords[1]][piece_coords[0]] = 0
    return state_copy

In [5]:
# We'll solve it by performing a breadth first search in two directions 
# starting at both the starting position and the goal.
# We'll stop after the two traversals meet.

# The state of each of the traversals.  There's a list of states to visit, 
# and a dictionary with the states already visited or to be visited.
directions = [
    {
        'to_visit': [str(starting)],
        'visited': {str(starting):True}
    },
    {
        'to_visit': [str(goal)],
        'visited': {str(goal):True}
    }
]

# The current direction to traverse.  
# This will swap between 0 and 1 to make sure that the traversals are balanced.
direction = 0

# An indication whether the traversals have met.
complete = False

# We'll maintain a graph representation too so we can retrace the steps at the end.
g = nx.DiGraph()

# Keep traversing while the traversals haven't met, or their are states to visit.
while (not complete) or (len(directions[0]['to_visit']) > 0) or (len(directions[1]['to_visit']) > 0):
    
    # Swap the traversal being processed
    direction = not direction
    
    # Get the visited and to_visit objects.
    to_visit = directions[direction]['to_visit']
    visited = directions[direction]['visited']
    
    # If to_visit is empty, continue
    if not len(to_visit):
        continue
    
    # Pop the first state and convert it to a numpy array to operate on.
    current_str = to_visit.pop(0)
    current = np.array(eval(current_str))
    
    # Get the location of the empty space (the location of the 0 in our array).
    space = (np.where(current == 0)[1][0], np.where(current == 0)[0][0])
    
    # Check each adjacent square to see if it can be moved into the empty space.
    for x,y,d in adjacent(space):
        # If their are no pieces blocking the movement, then it can be moved.
        if len(df[(df['moving_piece'] == current[y][x]) & (df['direction'] == d)].merge(
            relative_locations(space, current), on=['static_piece', 'x', 'y']
        )) == 0:
            
            # Apply the movement and get the new state then add the state transition to our graph.
            new_str = str(apply_move(current.tolist(), space, (x,y)))
            g.add_edge(current_str, new_str, move=(current[y][x], d))
            
            # If the state hasn't already been found, add it to the to_visit list.
            # If the traversals have already met, then don't add it.
            if new_str not in visited:
                if not complete:
                    to_visit.append(new_str)
                visited[new_str] = True
                
                # Check if the new state has been visited by the other traversal.
                # If so, then we're almost complete and just need to clean up the 
                # remaining edges from the to_vist lists.
                if new_str in directions[not direction]['visited']:
                    complete = True

In [6]:
# Get the shortest path and print out the step number and movement

dirs = {'l': 'left', 'r': 'right', 'u': 'up', 'd': 'down'}

path = nx.shortest_path(g, str(starting), str(goal))
for i in range(len(path)-1):
    p, d = g.get_edge_data(path[i], path[i+1])['move']
    print("%s: move %s %s" % (i+1, p, dirs[d]))

1: move 12 down
2: move 8 down
3: move 7 right
4: move 6 right
5: move 2 down
6: move 1 right
7: move 5 up
8: move 9 up
9: move 10 left
10: move 14 up
11: move 15 left
12: move 11 down
13: move 14 right
14: move 2 down
15: move 1 down
16: move 3 left
17: move 6 up
18: move 14 up
19: move 11 up
20: move 15 right
21: move 2 down
22: move 10 right
23: move 9 down
24: move 5 down
25: move 3 left
26: move 6 left
27: move 14 up
28: move 1 right
29: move 6 down
30: move 14 left
31: move 1 up
32: move 7 left
33: move 8 up
34: move 12 up
